In [1]:
from src.retriever.encode.kid import KidEncoder
encoder = KidEncoder()

/home/link/miniconda3/envs/conlearn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from transformers import AutoModel
from transformers import AutoTokenizer

In [11]:
model_path = '/home/link/spaces/LinhCSE/models/retriever'
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2')
model = AutoModel.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at /home/link/spaces/LinhCSE/models/retriever were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'mlp.dense.bias', 'lm_head.layer_norm.weight', 'mlp.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/link/spaces/LinhCSE/models/retriever and are newly i

In [12]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [13]:
def encode(texts, max_seq_len=256, return_output='np', **kwargs):
    features = tokenizer(
        texts,
        max_length=max_seq_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        pad_to_multiple_of=256,
    )
    with torch.no_grad():
        inputs = {
            'input_ids': features['input_ids'].to(model.device),
            'attention_mask': features['attention_mask'].to(model.device),
            'token_type_ids': features['token_type_ids'].to(model.device),
        }
        embedding = model(**inputs)
        embedding = embedding.last_hidden_state[:, 0, :]  # CLS token
        # embedding = embedding.last_hidden_state.mean(dim=1)  # AVG token

    if return_output == 'np':
        embedding.detach().cpu().numpy()
    return embedding

In [3]:
import json
from tqdm import tqdm 
from pyvi import ViTokenizer

In [18]:
with open('/home/link/spaces/LinhCSE/data/corpus.json') as f:
    corpus = json.load(f)

In [5]:
from src.retriever.utils.normalize import normalize_encode
from src.retriever.utils.normalize import normalize_word_diacritic
from src.retriever.utils.normalize import remove_punctuation
def pre_process(text):
    text = normalize_encode(normalize_word_diacritic(text))
    text = text.lower()
    text = remove_punctuation(text)
    return text

/home/link/miniconda3/envs/conlearn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with torch.no_grad():
    for document in tqdm(corpus):
        for chunk in document['section']:
            text = ViTokenizer.tokenize(chunk['content'])
            text = pre_process(text)
        
            chunk["embedding"] = encode(text)

In [6]:
all_texts = []
for document in tqdm(corpus):
    for chunk in document['section']:
        text = ViTokenizer.tokenize(chunk['content'])
        text = pre_process(text)
        all_texts.append(text)

100%|██████████| 3205/3205 [04:07<00:00, 12.94it/s]


In [17]:
import numpy as np
embedding_query = None
for i in tqdm(range(0, len(all_texts), 512)):
    # for query in all_texts[i: i + 64]:
        queries = all_texts[i: i + 512]
        # queries.append(query)

        embedding = encode(queries)
        if embedding_query is None:
            embedding_query = embedding.detach().cpu().numpy()
        else:
            embedding_query = np.append(
                embedding_query,
                embedding.detach().cpu().numpy(),
                axis=0,
            )

100%|██████████| 765/765 [31:37<00:00,  2.48s/it]


In [20]:
embedding_query.shape

(391391, 768)

In [ ]:
with torch.no_grad():
    for document in tqdm(corpus):
        for chunk in document['section']:
            text = ViTokenizer.tokenize(chunk['content'])
            text = pre_process(text)

            chunk["embedding"] = embedding_query[0]
            embedding_query = embedding_query[1:]